In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


2025-07-17 13:07:32.809903: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-17 13:07:32.810719: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 13:07:32.815854: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 13:07:32.827576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752782852.846871   59943 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752782852.85

# Preprocessing and Preparation

In [2]:
data_dir = 'data/RecyclableAndHouseholdWasteClassification/images'

image_size = (128, 128) # also try training using 256x256 images
batch_size = 32 # change this to 16, 32, 64, or 128 and compare the results

"""
32: Validation Loss: 1.9282389879226685, Validation Accuracy: 0.6756666898727417
64: Validation Loss: 1.9139728546142578, Validation Accuracy: 0.6703333258628845
"""


datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# load train data (the 80% part of the split)

train_data = datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='training')

valid_data = datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='validation')

Found 12000 images belonging to 30 classes.
Found 3000 images belonging to 30 classes.


# Build the Model

In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(len(train_data.class_indices), activation='softmax'))

/home/adi/Congressional-App-Challenge-Backend/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-17 13:07:37.399503: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Compile the Model

In [4]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the Model

In [5]:
# i hope this doesn't explode

# Train the model
history = model.fit(
    train_data,
    epochs=25,  # play with this number to maximize accuracy
    validation_data=valid_data
)


/home/adi/Congressional-App-Challenge-Backend/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 45s 118ms/step - accuracy: 0.1598 - loss: 3.1381 - val_accuracy: 0.4233 - val_loss: 2.0427
Epoch 2/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 43s 114ms/step - accuracy: 0.5556 - loss: 1.6036 - val_accuracy: 0.5967 - val_loss: 1.4996
Epoch 3/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - accuracy: 0.7927 - loss: 0.7498 - val_accuracy: 0.6490 - val_loss: 1.4459
Epoch 4/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 45s 120ms/step - accuracy: 0.9054 - loss: 0.3604 - val_accuracy: 0.6783 - val_loss: 1.4514
Epoch 5/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 113ms/step - accuracy: 0.9462 - loss: 0.2160 - val_accuracy: 0.6690 - val_loss: 1.6131
Epoch 6/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 46s 121ms/step - accuracy: 0.9608 - loss: 0.1549 - val_accuracy: 0.6970 - val_loss: 1.6067
Epoch 7/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 113ms/step - accuracy: 0.9660 - loss: 0.1292 - val_accuracy: 0.6913 - val_loss: 1.5962
Epoch 8/25
375/375 ━━━━━━━━━━━━━━━━━━━━ 47s 125ms/step - accuracy: 0.9671 - loss: 0

In [6]:
val_loss, val_acc = model.evaluate(valid_data)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')

94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.6741 - loss: 2.2752
Validation Loss: 2.423795461654663, Validation Accuracy: 0.6620000004768372
